Resolución completa del problema de Flujo de Stokes en una cavidad con pared móvil, listo para copiar y pegar en un Jupyter Notebook.

-----

### 1\. Inicialización y parámetros

Esta celda configura todas las librerías necesarias y los parámetros del problema.

```python
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# --- Parámetros de la simulación ---
nx, ny = 41, 41  # Número de puntos en cada dirección
L = 1.0          # Longitud de la cavidad cuadrada
dx = L / (nx - 1)
dy = L / (ny - 1)

# Parámetros del método iterativo
omega = 1.7 # Factor de relajación (típico para SOR)
l1_target = 1e-6 # Criterio de convergencia (norma L1)

# --- Funciones auxiliares ---

def l1_norm(u, u_ref):
    """Calcula la norma L1 de la diferencia entre dos arrays."""
    diff = np.sum(np.abs(u - u_ref))
    return diff

def plot_contour(data, title):
    """Grafica los contornos de un array 2D."""
    fig, ax = plt.subplots(figsize=(8, 8))
    x = np.linspace(0, L, nx)
    y = np.linspace(0, L, ny)
    X, Y = np.meshgrid(x, y)
    
    # Crea los contornos
    levels = np.linspace(np.min(data), np.max(data), 20)
    ax.contourf(X, Y, data, levels=levels, cmap=cm.viridis)
    ax.set_title(title)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', adjustable='box')
    plt.show()
```

-----

### 2\. Definición del problema y solución

Aquí se implementan los métodos iterativos para resolver las ecuaciones de vorticidad-función de corriente. El problema se resuelve de forma iterativa hasta que se alcanza la convergencia.

```python
# --- Ecuaciones discretas ---
# Ecuación de la vorticidad:
# omega_i,j = 0.25 * (omega_i+1,j + omega_i-1,j + omega_i,j+1 + omega_i,j-1)

# Ecuación de la función de corriente:
# psi_i,j = 0.25 * (psi_i+1,j + psi_i-1,j + psi_i,j+1 + psi_i,j-1 - (dx**2) * omega_i,j)

# --- Inicialización de las matrices ---
psi = np.zeros((nx, ny)) # Función de corriente
omega = np.zeros((nx, ny)) # Vorticidad

# --- Bucle de la simulación ---
l1_diff_psi = 1.0
l1_diff_omega = 1.0
iterations = 0

print("Comenzando el cálculo...")
start_time = time.time()

while l1_diff_psi > l1_target or l1_diff_omega > l1_target:
    
    # Guarda copias de las matrices para el cálculo de la diferencia
    psi_n = psi.copy()
    omega_n = omega.copy()
    
    # --- Actualiza la vorticidad (omega) ---
    # Aplica el método de Gauss-Seidel
    for j in range(1, ny - 1):
        for i in range(1, nx - 1):
            omega[i, j] = 0.25 * (omega[i+1, j] + omega[i-1, j] + omega[i, j+1] + omega[i, j-1])
            
    # Aplica las condiciones de borde para la vorticidad
    # omega en la pared superior
    omega[:, -1] = -2.0 * (psi[:, -2] - psi[:, -1]) / dy**2 - 2.0 / dy
    # omega en el resto de las paredes (basado en derivadas = 0)
    omega[:, 0] = -2.0 * (psi[:, 1] - psi[:, 0]) / dy**2
    omega[0, :] = -2.0 * (psi[1, :] - psi[0, :]) / dx**2
    omega[-1, :] = -2.0 * (psi[-2, :] - psi[-1, :]) / dx**2

    # --- Actualiza la función de corriente (psi) ---
    # Aplica el método SOR (sobre-relajación sucesiva)
    for j in range(1, ny - 1):
        for i in range(1, nx - 1):
            psi_temp = 0.25 * (psi[i+1, j] + psi[i-1, j] + psi[i, j+1] + psi[i, j-1] - (dx**2) * omega[i, j])
            psi[i, j] = (1 - omega) * psi[i, j] + omega * psi_temp

    # Mantiene las condiciones de borde (psi = 0 en las paredes fijas)
    psi[0, :] = 0
    psi[-1, :] = 0
    psi[:, 0] = 0
    psi[:, -1] = 0
    
    # Calcula la diferencia para verificar la convergencia
    l1_diff_psi = l1_norm(psi, psi_n)
    l1_diff_omega = l1_norm(omega, omega_n)
    iterations += 1
    
    if iterations % 100 == 0:
        print(f"Iteración: {iterations}, L1_psi: {l1_diff_psi:.6e}, L1_omega: {l1_diff_omega:.6e}")

end_time = time.time()
print("\n--- Simulación terminada ---")
print(f"Número total de iteraciones: {iterations}")
print(f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")
```

-----

### 3\. Visualización de los resultados

Finalmente, esta celda grafica los resultados de la función de corriente (`psi`) para visualizar el campo de flujo final.

```python
# Muestra los contornos de la función de corriente
plot_contour(psi, 'Campo de flujo de la función de corriente (psi)')
```